In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import  scipy.signal

time_arr =  np.arange(0, 10, 1/16000)


signal = np.sin(2*np.pi*200*time_arr)+np.sin(2*np.pi*2500*time_arr)+np.sin(2*np.pi*5500*time_arr)
stft = np.abs(scipy.signal.stft(signal, fs=16000.0, window='hann', nperseg=512, noverlap=512-128)[2])
print(stft.dtype)
plt.imshow(stft)
plt.show()

fp = np.memmap('data/test_2.dat', dtype='float64', mode='w+', shape=stft.shape)
fp[:] = stft[:]
del fp

In [ ]:
import torch
torch.cuda.init()
from torch import nn

from model.borderEncoder import BorderEncoder
from model.discriminator import Discriminator
from model.generator import Generator

import time

from utils.colorize import colorize
import torch.autograd as autograd


def calc_gradient_penalty_bayes(discriminator, real_data, fake_data, gamma):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    batch_size = real_data.size()[0]

    alpha = torch.rand(batch_size, 1, 1, 1)
    alpha = alpha.expand(real_data.size()).to(device)

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    interpolates = autograd.Variable(interpolates, requires_grad=True).to(device)

    disc_interpolates = discriminator(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).to(device),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
#     gradients = gradients.view(real)
    print(gradients.norm(2, dim=[1,2,3]).mean())
    gradient_penalty = ((gradients.norm(2, dim=[1,2,3]) - 1) ** 2).mean() * gamma

    return gradient_penalty


__author__ = 'Andres'


def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.0)


def train(args, device, train_loader, epoch, summary_writer):
    discriminators = nn.ModuleList(
            [Discriminator(args['discriminator'], args['discriminator_in_shape']) for _ in range(3)]
        ).to(device)
    discriminators.apply(init_weights)

    left_border_encoder = BorderEncoder(args['borderEncoder']).to(device)
    right_border_encoder = BorderEncoder(args['borderEncoder']).to(device)
    left_border_encoder.apply(init_weights)
    right_border_encoder.apply(init_weights)

    generator = Generator(args['generator'], args['generator_input']).to(device)
    generator.apply(init_weights)

    optim_g = torch.optim.Adam(list(generator.parameters()) + list(left_border_encoder.parameters()) +
                               list(right_border_encoder.parameters()),
                               lr=args['optimizer']['generator']['learning_rate'],
                               betas=(0.5, 0.9))
    optims_d = [torch.optim.Adam(discriminator.parameters(),
                               lr=args['optimizer']['discriminator']['learning_rate'],
                               betas=(0.5, 0.9)) for discriminator in discriminators]
    # try:
    # left_border_encoder.train()
    # right_border_encoder.train()
    # generator.train()
    # discriminators.train()
    print('try')
    start_time = time.time()
    prev_iter_time = start_time
    # train_loader = tqdm.tqdm(train_loader)
    for batch_idx, data in enumerate(train_loader):
        print(batch_idx)
        data = data.to(device).float()
        data = data.view(args['optimizer']['batch_size'], *args['spectrogram_shape'])
        real_spectrograms = data[::2]
        fake_left_borders = data[1::2, :, :, :args['split'][0]]
        fake_right_borders = data[1::2, :, :, args['split'][0] + args['split'][1]:]

        #optimize D

        for _ in range(args['optimizer']['n_critic']):
            for index, (discriminator, optim_d) in enumerate(zip(discriminators, optims_d)):
                optim_d.zero_grad()
                print(index)
                encoded_left_border = left_border_encoder(fake_left_borders)
                encoded_right_border = right_border_encoder(fake_right_borders)
                encoded_size = encoded_left_border.size()
                noise = torch.rand(encoded_size[0], 4, encoded_size[2], encoded_size[3]).to(device)
                generated_spectrograms = generator(torch.cat((encoded_left_border, encoded_right_border, noise), 1))

                fake_spectrograms = torch.cat((fake_left_borders, generated_spectrograms, fake_right_borders), 3)
                scale = 2**index
                time_axis = args['spectrogram_shape'][2]
                start = int((time_axis - (time_axis // 4) * scale) / 2)
                end = time_axis - start
                x_fake = fake_spectrograms[:, :, :, start:end:scale]
                x_real = real_spectrograms[:, :, :, start:end:scale]

                d_loss_r = torch.mean(discriminator(x_real))
                d_loss_f = torch.mean(discriminator(x_fake))

                grad_pen = calc_gradient_penalty_bayes(discriminator, x_real, x_fake, args['gamma_gp'])
                d_loss_gp = torch.mean(grad_pen)
                disc_loss = d_loss_f - d_loss_r + d_loss_gp
                print(d_loss_gp)

                disc_loss.backward()
                optim_d.step()

#             grad_pen_2 = wgan_regularization(discriminators, x_real, x_fake, args['gamma_gp'])
#             d_loss_gp_2 += grad_pen_2.mean()
#             grad_pen_3 = calc_gradient_penalty(discriminators, x_real, x_fake, args['gamma_gp'])
#             d_loss_gp_3 += grad_pen_3.mean()

#             print(d_loss_gp_2)
#             print(d_loss_gp_3)

        #optimize G

        optim_g.zero_grad()

        encoded_left_border = left_border_encoder(fake_left_borders)
        encoded_right_border = right_border_encoder(fake_right_borders)
        encoded_size = encoded_left_border.size()
        noise = torch.rand(encoded_size[0], 4, encoded_size[2], encoded_size[3]).to(device)
        generated_spectrograms = generator(torch.cat((encoded_left_border, encoded_right_border, noise), 1))

        fake_spectrograms = torch.cat((fake_left_borders, generated_spectrograms, fake_right_borders), 3)
        d_loss_f = 0

        for index, discriminator in enumerate(discriminators):
            scale = 2 ** index
            time_axis = args['spectrogram_shape'][2]
            start = int((time_axis - (time_axis // 4) * scale) / 2)
            end = time_axis - start
            x_fake = fake_spectrograms[:, :, :, start:end:scale]
            d_loss_f += torch.mean(discriminator(x_fake))

        gen_loss = - d_loss_f
        gen_loss.backward()
        optim_g.step()

#         if batch_idx % args['log_interval'] == 0:
#             current_time = time.time()

#             print(" * Epoch: [{:2d}] [{:4d}/{:4d} ({:.0f}%)] "
#                   "Counter:{:2d}\t"
#                   "({:4.1f} min\t"
#                   "{:4.3f} examples/sec\t"
#                   "{:4.2f} sec/batch)\n"
#                   "   Disc batch loss:{:.8f}\t"
#                   "   Gen batch loss:{:.8f}\t"
#                   "   Reg batch :{:.8f}\t".format(
#                 int(epoch),
#                 int(batch_idx*len(data)),
#                 int(len(train_loader.dataset)/len(data)), 100. * batch_idx / len(train_loader), int(batch_idx),
#                 (current_time - start_time) / 60,
#                 args['log_interval'] * args['optimizer']['batch_size'] / (current_time - prev_iter_time),
#                 (current_time - prev_iter_time) / args['log_interval'],
#                 disc_loss.item(),
#                 gen_loss.item(),
#                 d_loss_gp.item()))
#             prev_iter_time = current_time
#         if batch_idx % args['tensorboard_interval'] == 0:
#             summary_writer.add_scalar("Disc/Neg_Loss", -disc_loss, global_step=batch_idx)
#             summary_writer.add_scalar("Disc/Neg_Critic", d_loss_f - d_loss_r, global_step=batch_idx)
#             summary_writer.add_scalar("Disc/Loss_f", d_loss_f, global_step=batch_idx)
#             summary_writer.add_scalar("Disc/Loss_r", d_loss_r, global_step=batch_idx)
#             summary_writer.add_scalar("Gen/Loss", gen_loss, global_step=batch_idx)

#             for index in range(4):
#                 summary_writer.add_image("images/Real_Image/" + str(index), colorize(real_spectrograms[index]), global_step=batch_idx)
#                 summary_writer.add_image("images/Fake_Image/" + str(index), colorize(fake_spectrograms[index], -1, 1), global_step=batch_idx)
#                 # except Exception as e:
            # print(e)


In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter

from data.trainDataset import TrainDataset
import logging

# logging.getLogger().setLevel(logging.DEBUG)  # set root logger to debug

"""Just so logging works..."""
formatter = logging.Formatter('%(name)s:%(levelname)s:%(message)s')
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
console_handler.setFormatter(formatter)
logging.getLogger().addHandler(console_handler)
"""Just so logging works..."""

__author__ = 'Andres'


signal_split = [192, 128, 192]
md = 4

params_discriminator = dict()
params_discriminator['stride'] = [2,2,2,2,2]
params_discriminator['nfilter'] = [md, 2*md, 4*md, 8*md, 16*md]
params_discriminator['shape'] = [[5, 5], [5, 5], [5, 5], [5, 5], [5, 5]]
params_discriminator['full'] = []
params_discriminator['minibatch_reg'] = False
params_discriminator['summary'] = True
params_discriminator['data_size'] = 2
params_discriminator['apply_phaseshuffle'] = True
params_discriminator['spectral_norm'] = True

params_generator = dict()
params_generator['stride'] = [2, 2, 2, 2, 2]
params_generator['latent_dim'] = 100
params_generator['nfilter'] = [8*md, 4*md, 2*md, md, 1]
params_generator['shape'] = [[4, 4], [4, 4], [8, 8], [8, 8], [8, 8]]
params_generator['padding'] = [[1, 1], [1, 1], [3, 3], [3, 3], [3, 3]]
params_generator['full'] = 256*md
params_generator['summary'] = True
params_generator['data_size'] = 2
params_generator['spectral_norm'] = True
params_generator['in_conv_shape'] = [8, 4]
params_generator['borders'] = dict()
params_generator['borders']['nfilter'] = [md, 2*md, md, md/2]
params_generator['borders']['shape'] = [[5, 5],[5, 5],[5, 5],[5, 5]]
params_generator['borders']['stride'] = [2, 2, 3, 4]
params_generator['borders']['data_size'] = 2
# This does not work because of flipping, border 2 need to be flipped tf.reverse(l, axis=[1]), ask Nathanael
params_generator['borders']['width_full'] = None


# Optimization parameters inspired from 'Self-Attention Generative Adversarial Networks'
# - Spectral normalization GEN DISC
# - Batch norm GEN
# - TTUR ('GANs Trained by a Two Time-Scale Update Rule Converge to a Local Nash Equilibrium')
# - ADAM  beta1=0 beta2=0.9, disc lr 0.0004, gen lr 0.0001
# - Hinge loss
# Parameters are similar to the ones in those papers...
# - 'PROGRESSIVE GROWING OF GANS FOR IMPROVED QUALITY, STABILITY, AND VARIATION'
# - 'LARGE SCALE GAN TRAINING FOR HIGH FIDELITY NATURAL IMAGE SYNTHESIS'
# - 'CGANS WITH PROJECTION DISCRIMINATOR'

params_optimization = dict()
params_optimization['batch_size'] = 64*2
params_discriminator['batch_size'] = 64*2

params_optimization['epoch'] = 600
params_optimization['n_critic'] = 2
params_optimization['generator'] = dict()
params_optimization['generator']['optimizer'] = 'adam'
params_optimization['generator']['kwargs'] = [0.5, 0.9]
params_optimization['generator']['learning_rate'] = 1e-4
params_optimization['discriminator'] = dict()
params_optimization['discriminator']['optimizer'] = 'adam'
params_optimization['discriminator']['kwargs'] = [0.5, 0.9]
params_optimization['discriminator']['learning_rate'] = 1e-4

# all parameters
params = dict()
params['net'] = dict() # All the parameters for the model
params['net']['generator'] = params_generator
params['net']['discriminator'] = params_discriminator
params['net']['prior_distribution'] = 'gaussian'
params['net']['shape'] = [1, 256, 128*4] # Shape of the image
params['net']['inpainting']=dict()
params['net']['inpainting']['split']=signal_split
params['net']['gamma_gp'] = 10 # Gradient penalty
# params['net']['fs'] = 16000//downscale
params['net']['loss_type'] ='wasserstein'

params['optimization'] = params_optimization
params['summary_every'] = 250 # Tensorboard summaries every ** iterations
params['print_every'] = 50 # Console summaries every ** iterations
params['save_every'] = 1000 # Save the model every ** iterations
# params['summary_dir'] = os.path.join(global_path, name +'_summary/')
# params['save_dir'] = os.path.join(global_path, name + '_checkpoints/')
params['Nstats'] = 500

args = dict()
args['generator'] = params_generator
args['discriminator'] = params_discriminator
args['borderEncoder'] = params_generator['borders']
args['discriminator_in_shape'] = [1, 256, 128]
args['generator_input'] = 2*6*4*2+24*4
args['optimizer'] = params_optimization
args['split'] = signal_split
args['log_interval'] = 50
args['spectrogram_shape'] = params['net']['shape']
args['gamma_gp'] = params['net']['gamma_gp']
args['tensorboard_interval'] = 250

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

examples_per_file = 16
trainDataset = TrainDataset("data/", window_size=512, examples_per_file=examples_per_file)

train_loader = torch.utils.data.DataLoader(trainDataset,
    batch_size=args['optimizer']['batch_size']//examples_per_file, shuffle=True)#,
                                           #num_workers=1, drop_last=True)


experiment_name = 'pytorch'
# summary_writer = SummaryWriter('saved_results/' + experiment_name)

# for epoch in range(10):
train(args, device, train_loader, epoch, 2)
